In [1]:
%cd ..

/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib nbagg
import os.path as op
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau
from keras.models import load_model
from keras_tqdm import TQDMNotebookCallback
from tqdm import tqdm_notebook

from data import ZeroFilled2DSequence
from unet import unet

/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/venv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
# paths
train_path = '/media/Zaccharie/UHRes/singlecoil_train/singlecoil_train/'
val_path = '/media/Zaccharie/UHRes/singlecoil_val/'
test_path = '/media/Zaccharie/UHRes/singlecoil_test/'

In [4]:
n_samples_train = 34742
n_samples_val = 7135

n_volumes_train = 973
n_volumes_val = 199

In [5]:
# generators
AF = 4
train_gen = ZeroFilled2DSequence(train_path, af=AF, norm=True)
val_gen = ZeroFilled2DSequence(val_path, mode='validation', af=AF, norm=True)

In [6]:
run_params = {
    'n_layers': 4, 
    'pool': 'max', 
    "layers_n_channels": [32, 64, 128, 256], 
    'layers_n_non_lins': 2,
}
n_epochs = 100
run_id = 'unet_wo_lastrelu_af4'
chkpt_path = f'checkpoints/{run_id}' + '-{epoch:02d}.hdf5'

In [7]:
chkpt_cback = ModelCheckpoint(chkpt_path, period=50)
log_dir = op.join('logs', run_id)
tboard_cback = TensorBoard(
    log_dir=log_dir, 
    histogram_freq=0, 
    write_graph=True, 
    write_images=False, 
)
lr_on_plat_cback = ReduceLROnPlateau(monitor='val_loss', min_lr=5*1e-5, mode='auto', patience=3)
# fix from https://github.com/bstriner/keras-tqdm/issues/31#issuecomment-488453557
tqdm_cb = TQDMNotebookCallback()
setattr(tqdm_cb,'on_train_batch_begin',lambda x,y:None)
setattr(tqdm_cb,'on_train_batch_end',lambda x,y:None)

In [8]:
model = unet(input_size=(320, 320, 1), lr=1e-3, **run_params)
print(model.summary())

W0730 11:40:20.487751 140259640231680 deprecation_wrapper.py:119] From /volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/venv/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0730 11:40:20.508064 140259640231680 deprecation_wrapper.py:119] From /volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/venv/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0730 11:40:20.513346 140259640231680 deprecation_wrapper.py:119] From /volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/venv/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4185: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W0730 11:40:20.537530 140259640231680 deprecation_wrapper.py:119] From /volatile/home/Zacchar

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 320, 320, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 320, 320, 32) 320         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 320, 320, 32) 9248        conv2d_1[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 160, 160, 32) 0           conv2d_2[0][0]                   
__________________________________________________________________________________________________
conv2d_3 (

In [9]:
model.fit_generator(
    train_gen, 
    steps_per_epoch=n_volumes_train, 
    epochs=n_epochs,
    validation_data=val_gen,
    validation_steps=n_volumes_val,
    verbose=0,
    callbacks=[tqdm_cb, tboard_cback, chkpt_cback, lr_on_plat_cback],
    max_queue_size=50,
    use_multiprocessing=True,
    workers=35,
)

In [10]:
# # overfitting trials

# data = train_gen[0]
# val_data = val_gen[0]
# model.fit(
#     x=data[0], 
#     y=data[1], 
# #     validation_data=val_data, 
#     batch_size=data[0].shape[0], 
#     epochs=50, 
#     verbose=2, 
#     shuffle=False,
# )

W0730 11:40:23.671902 140259640231680 deprecation_wrapper.py:119] From /volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/venv/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W0730 11:40:24.145766 140259640231680 deprecation_wrapper.py:119] From /volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/venv/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:973: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.



Epoch 1/50
 - 9s - loss: 1.4108 - mean_squared_error: 4.2043 - keras_psnr: 25.0042 - keras_ssim: -1.0433e-01
Epoch 2/50
 - 1s - loss: 8.3795 - mean_squared_error: 103.3717 - keras_psnr: 11.0815 - keras_ssim: -2.0473e-02
Epoch 3/50
 - 1s - loss: 1.7845 - mean_squared_error: 5.7691 - keras_psnr: 23.7973 - keras_ssim: 0.0691
Epoch 4/50
 - 1s - loss: 0.5254 - mean_squared_error: 0.6258 - keras_psnr: 34.4398 - keras_ssim: 0.6909
Epoch 5/50
 - 1s - loss: 0.5675 - mean_squared_error: 0.6494 - keras_psnr: 33.9805 - keras_ssim: 0.6394
Epoch 6/50
 - 1s - loss: 0.6040 - mean_squared_error: 0.7701 - keras_psnr: 33.4162 - keras_ssim: 0.6137
Epoch 7/50
 - 1s - loss: 0.5159 - mean_squared_error: 0.6111 - keras_psnr: 34.5128 - keras_ssim: 0.6842
Epoch 8/50
 - 1s - loss: 0.4977 - mean_squared_error: 0.5603 - keras_psnr: 34.8786 - keras_ssim: 0.7117
Epoch 9/50
 - 1s - loss: 0.5066 - mean_squared_error: 0.5632 - keras_psnr: 34.8036 - keras_ssim: 0.7066
Epoch 10/50
 - 1s - loss: 0.4960 - mean_squared_erro